In [ ]:
s_file = "/Users/denismottet/Documents/GitHub/NeuArm-DataAnalysis/data/AgePie/AgePie_A16.snirf"
delay = 6985
duration = 8105

In [ ]:
# Load the snirf file
import mne

raw = mne.io.read_raw_snirf(s_file, preload=True, verbose="CRITICAL")
raw.close()

raw_data = raw.get_data()
print(raw_data.shape)
# plot the raw data (to see the markers-annotations)
raw.plot()

# close the file 
raw.close()


# get the annotations (there is no Events in this file)
annotations = raw.annotations

# print all annotations
for i in range(len(annotations.description)):
    e = annotations.description[i]
    t = annotations.onset[i]
    # print("{:2d}: {:10.3f}s = {} ".format(i, t, e))

# find the first annotation containing the word 111
iStart = -1
for i in range(len(annotations.description)):
    if "111" in annotations.description[i]:
        iStart = i
        break
print(
    "First event 111 is at index {} with label {} at time {}".format(
        iStart, annotations.description[iStart], annotations.onset[iStart]
    ),
)

# find all annotations containing the word 111
i111s = []
for i in range(len(annotations.description)):
    if "111" in annotations.description[i]:
        i111s.append(i)
print("All events 111 are at indexes {}".format(i111s))

In [ ]:
from snirf import Snirf
import numpy as np
import copy 

snirf = Snirf(s_file)

# get the nirs time series 
nirs_data = snirf.nirs[0].data[0].dataTimeSeries
nirs_time = snirf.nirs[0].data[0].time
nirs_meas_list = snirf.nirs[0].data[0].measurementList
n_channels = len(nirs_meas_list)
n_samples = len(nirs_time)
print("n_channels = {}, n_samples = {}".format(n_channels, n_samples))


# metaDataTags
print("date = {}".format(snirf.nirs[0].metaDataTags.MeasurementDate))
print("time = {}".format(snirf.nirs[0].metaDataTags.MeasurementTime))
print("Subject  = {}".format(snirf.nirs[0].metaDataTags.SubjectID))

# probe
print("sourcePos2D 2D = {}".format(snirf.nirs[0].probe.sourcePos2D))
print("detectorPos2D 2D = {}".format(snirf.nirs[0].probe.detectorPos2D))

print("nirs_data.shape = {}".format(nirs_data.shape))

# get the names of the channels
channel_names = []
n_sources = len(snirf.nirs[0].probe.sourceLabels)
n_detectors = len(snirf.nirs[0].probe.detectorLabels)
n_wavelengths = len(snirf.nirs[0].probe.wavelengths)
print("n_sources = {}, n_detectors = {}, n_wavelengths = {}".format(n_sources, n_detectors, n_wavelengths))

for w in range(n_wavelengths):
    for s in range(n_sources):
        d = int(np.floor_divide(s , n_sources / n_detectors))
        channel_names.append(
            "{}-{}-{:3.0f}".format(
                snirf.nirs[0].probe.sourceLabels[s],
                snirf.nirs[0].probe.detectorLabels[d],
                snirf.nirs[0].probe.wavelengths[w],
            )
        )

def print_annotations(annotations):
    n_annotations = len(annotations)
    for a in range(n_annotations):
        if annotations[a].data.ndim == 1:
            print("{} = {}".format(annotations[a].name, annotations[a].data))
        else:
            for i in range(len(annotations[a].data)):
                print("{} = {}".format(annotations[a].name, annotations[a].data[i]))

# print the annotations
print("annotations")
print_annotations(snirf.nirs[0].stim)

# keep only the annotations that are in the time window
annotations = snirf.nirs[0].stim
n_annotations = len(annotations)

beg = snirf.nirs[0].data[0].time[delay]
end = snirf.nirs[0].data[0].time[delay + duration]
print("beg = {}, end = {}".format(beg, end))

# modify the time of the annotations
for a in range(n_annotations):
    if annotations[a].data.ndim == 1:
        annotations[a].data[0] = annotations[a].data[0] - beg
    else:
        for i in range(len(annotations[a].data)):
            annotations[a].data[i][0] = annotations[a].data[i][0] - beg


# print the new annotations
print("new annotations")
print_annotations(snirf.nirs[0].stim)


# plot the nirs time series
import matplotlib.pyplot as plt

plt.figure()
plt.plot(nirs_time, nirs_data, linewidth=0.5, label=channel_names)
plt.title("nirs time series")
plt.xlabel("time (s)")
plt.ylabel("nirs")
plt.legend()
plt.show()


# Moditfy the file 


In [ ]:
new_fname = "new_AgePie_A16_2.snirf"
# NOTE: modifying the data in place will modify the original file
snirf.save(new_fname) # before modifying the data

new_nirs_data = nirs_data[delay:delay+duration,:]
new_nirs_time = nirs_time[delay:delay+duration]

new_fname = "new_AgePie_A16_2.snirf"
snirf.save(new_fname) # before modifying the data

snirf.nirs[0].data[0].dataTimeSeries = new_nirs_data
snirf.nirs[0].data[0].time = new_nirs_time

snirf.save()

# Load the file and plot the data

In [ ]:
new = mne.io.read_raw_snirf(new_fname, preload=True, verbose="CRITICAL")


new.plot()
